In [2]:
import cv2
import numpy as np
import os

### bounding box format

In [25]:
# Load the image
image = cv2.imread("datatrain5.jpg")

# Load the annotations
annotations = np.loadtxt("datatrain1.txt")

for i, annotation in enumerate(annotations):
    # Extract the class label and bounding box coordinates
    class_label, x_center, y_center, width, height = annotation

    # Calculate the bounding box coordinates in pixels
    x1 = int((x_center - width / 2) * image.shape[1])
    y1 = int((y_center - height / 2) * image.shape[0])
    x2 = int((x_center + width / 2) * image.shape[1])
    y2 = int((y_center + height / 2) * image.shape[0])

    # Crop the image using the bounding box coordinates
    cropped_image = image[y1:y2, x1:x2]

    # Save the cropped image
    cv2.imwrite(f"cropped_rectangle_image_{i}.jpg", cropped_image)

### polygon format

In [27]:
# Load the image
image = cv2.imread('datatrain5.jpg')

# Load the annotations
data = []
with open('datatrain5.txt', 'r') as f:
    for line in f:
        row = line.strip().split()
        data.append(row)

for i, d in enumerate(data):
    # Extract the class label and polygon coordinates
    class_label = int(data[i][0])
    polygon = np.array(data[i][1:], dtype=float).reshape(-1, 2)

    # Convert the polygon coordinates to integers
    polygon = np.round(polygon * np.array([image.shape[1], image.shape[0]])).astype(int)

    # Create a mask for the polygon
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [polygon], 255)

    # Find the bounding rectangle of the polygon
    x, y, w, h = cv2.boundingRect(polygon)

    # Crop the image using the bounding rectangle
    cropped_image = image[y:y+h, x:x+w]

    # Save the cropped image
    cv2.imwrite(f'cropped_poly_image_{i}.jpg', cropped_image)


### read both

In [6]:
def extract_annotations(img_path, annotation_path, output_path):
    # Load the annotations
    data = []
    with open(annotation_path, 'r') as f:
        for line in f:
            row = line.strip().split()
            data.append(row)

    for i, annotation in enumerate(data):
        # Check if the current row has 5 values
        if len(annotation) == 5:
            # Call the function to extract bounding box annotations
            extract_bbox(img_path, annotation, output_path, i)
        else:
            # Call the function to extract polygon annotations
            extract_poly(img_path, annotation, output_path, i)


def extract_bbox(image_path: str, annotation: 'List[str]', base_output_path: str, i: int) -> None:
    # Load the image
    image = cv2.imread(image_path)

    # Extract the class label and bounding box coordinates
    class_label, x_center, y_center, width, height = map(float, annotation)

    class_label = str(int(class_label))

    # Calculate the bounding box coordinates in pixels
    x1 = int((x_center - width / 2) * image.shape[1])
    y1 = int((y_center - height / 2) * image.shape[0])
    x2 = int((x_center + width / 2) * image.shape[1])
    y2 = int((y_center + height / 2) * image.shape[0])

    # Crop the image using the bounding box coordinates
    cropped_image = image[y1:y2, x1:x2]

    # Create the class label folder if it doesn't exist
    class_folder = os.path.join(base_output_path, class_label)
    if not os.path.exists(class_folder):
        os.makedirs(class_folder)

    base_name = os.path.basename(image_path)
    img_output = os.path.join(class_folder, f"{base_name}-rectangle-{i}.jpg")

    print(f"saved in {img_output}")

    # Save the cropped image
    cv2.imwrite(img_output, cropped_image)


def extract_poly(image_path: str, annotation: 'List[str]', base_output_path: str, i: int) -> None:
    # Load the image
    image = cv2.imread(image_path)

    # Extract the class label and polygon coordinates
    class_label = int(annotation[0])
    polygon = np.array(annotation[1:], dtype=float).reshape(-1, 2)

    # Convert the polygon coordinates to integers
    polygon = np.round(polygon * np.array([image.shape[1], image.shape[0]])).astype(int)

    # Create a mask for the polygon
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [polygon], 255)

    # Find the bounding rectangle of the polygon
    x, y, w, h = cv2.boundingRect(polygon)

    # Crop the image using the bounding rectangle
    cropped_image = image[y:y+h, x:x+w]

    # Create the class label folder if it doesn't exist
    class_folder = os.path.join(base_output_path, str(class_label))
    if not os.path.exists(class_folder):
        os.makedirs(class_folder)

    base_name = os.path.basename(image_path)
    img_output = os.path.join(class_folder, f"{base_name}-polygon-{i}.jpg")

    print(f"saved in {img_output}")

    # Save the cropped image
    cv2.imwrite(img_output, cropped_image)


In [4]:
img_path = "datatrain5.jpg"
annotation_path = "datatrain5.txt"

extract_annotations(img_path, annotation_path, "./test")

saved in ./test\11\datatrain5-polygon-0.jpg
saved in ./test\1\datatrain5-polygon-1.jpg
saved in ./test\0\datatrain5-polygon-2.jpg
saved in ./test\8\datatrain5-polygon-3.jpg
saved in ./test\9\datatrain5-polygon-4.jpg
saved in ./test\31\datatrain5-polygon-5.jpg
saved in ./test\13\datatrain5-polygon-6.jpg


## Extract All

In [7]:
error_list = []

# Set the base paths to the images and annotations folders
base_images_path = "./yolov8-dataset-v2/"
base_annotations_path = "./yolov8-dataset-v2/"

# Set the base path to the output folder
base_output_path = "./yolov8-dataset-v2-extracted/"

# Set the dataset splits
splits = ["train", "valid", "test"]

empty_labels = []
total_data = 0
total_empty = 0

# Loop through the dataset splits
for split in splits:
    # Set the paths to the images and annotations folders for the current split
    images_path = os.path.join(base_images_path, split, "images")
    annotations_path = os.path.join(base_annotations_path, split, "labels")

    # Set the path to the output folder for the current split
    output_path = os.path.join(base_output_path, split)

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    empty_label = []

    # Loop through the annotation files
    for i, filename in enumerate(os.listdir(annotations_path)):
        
        print(f'reading: {filename}')
        
        annotation_file = os.path.join(annotations_path, filename)
        image_file = os.path.join(images_path, filename.replace(".txt", ".jpg"))
        
        if os.stat(annotation_file).st_size == 0:
            empty_label.append(annotation_file)
            total_empty += 1
            continue
        else:
            extract_annotations(image_file, annotation_file, output_path)
            total_data += 1
        
    print(f"total empty label in {split}: {len(empty_label)}")
    print()
    empty_labels.append(empty_label)

print(f'total empty labels: {total_empty}')
print(f'total extracted data: {total_data}')

reading: DataTrain100_png.rf.121b2a445bd9cbf7f452554772df2558.txt
saved in ./yolov8-dataset-v2-extracted/train\10\DataTrain100_png.rf.121b2a445bd9cbf7f452554772df2558.jpg-rectangle-0.jpg
saved in ./yolov8-dataset-v2-extracted/train\11\DataTrain100_png.rf.121b2a445bd9cbf7f452554772df2558.jpg-rectangle-1.jpg
saved in ./yolov8-dataset-v2-extracted/train\31\DataTrain100_png.rf.121b2a445bd9cbf7f452554772df2558.jpg-rectangle-2.jpg
saved in ./yolov8-dataset-v2-extracted/train\32\DataTrain100_png.rf.121b2a445bd9cbf7f452554772df2558.jpg-rectangle-3.jpg
saved in ./yolov8-dataset-v2-extracted/train\2\DataTrain100_png.rf.121b2a445bd9cbf7f452554772df2558.jpg-rectangle-4.jpg
saved in ./yolov8-dataset-v2-extracted/train\3\DataTrain100_png.rf.121b2a445bd9cbf7f452554772df2558.jpg-rectangle-5.jpg
saved in ./yolov8-dataset-v2-extracted/train\0\DataTrain100_png.rf.121b2a445bd9cbf7f452554772df2558.jpg-rectangle-6.jpg
saved in ./yolov8-dataset-v2-extracted/train\1\DataTrain100_png.rf.121b2a445bd9cbf7f452554